In [1]:
import awswrangler as wr
import mlflow
import datetime

# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


## Entrenamiento de un modelo Random Forest con parámetros específicos para clasificación de estrellas.

In [2]:
mlflow_server = "http://localhost:5000"
#mlflow_server = "http://192.168.0.21:5000"

mlflow.set_tracking_uri(mlflow_server)

# %%
# Cargamos los datos para realizar nuestro entrenamiento
X_train = wr.s3.read_csv("s3://data/final/train/star_X_train.csv")
y_train = wr.s3.read_csv("s3://data/final/train/star_y_train.csv")

X_test = wr.s3.read_csv("s3://data/final/test/star_X_test.csv")
y_test = wr.s3.read_csv("s3://data/final/test/star_y_test.csv")

In [3]:
print(X_train)

         obj_ID     alpha     delta         u         g         r         i  \
0     -0.151403  1.506682 -1.229315 -0.027928 -0.042080 -0.968742 -0.945357   
1      1.721125 -1.471247 -1.433494 -0.000849  0.012195  0.704779  0.932464   
2     -1.097912  0.808174  0.425162  0.088323  0.073398  1.037008  0.767655   
3      1.844968  1.798912 -0.070862  0.002050  0.021201  0.218118  0.134464   
4      0.293375 -0.112060  0.167313 -0.060604 -0.047515 -0.691826 -0.472763   
...         ...       ...       ...       ...       ...       ...       ...   
69995  0.773724 -0.472021 -0.652333  0.036142  0.001115 -0.474954 -0.524982   
69996 -0.469664 -0.254866  0.802815 -0.072694 -0.037749 -0.290759 -0.096327   
69997 -0.641045  0.352595  1.751331  0.038174  0.049127  0.645290  0.447952   
69998 -0.151276  1.497554 -1.180462  0.051178  0.050648  0.532544  0.435954   
69999  1.643676  1.862628 -0.354669  0.012606  0.059742  0.715077  0.485265   

              z    run_ID   cam_col  field_ID  spec

In [4]:
from mlflow.models import infer_signature
from mlflow_aux import get_or_create_experiment
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [5]:
# Creamos el experimento (usa el mismo experimento existente)
experiment_id = get_or_create_experiment("Star Classification")
print(experiment_id)

run_name = "Train RF"

1


## Inicio del experimento

In [6]:
# Entrenamos el modelo Random Forest con los parámetros especificados
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name):
    
    # Definimos los parámetros del modelo
    rf_params = {
        "n_estimators": 200,
        "max_depth": 25
    }
    
    # Creamos y entrenamos el modelo
    model = RandomForestClassifier(
        n_estimators=rf_params["n_estimators"],
        max_depth=rf_params["max_depth"]
    )
    
    model = model.fit(X_train, y_train.to_numpy().ravel())
    
    # Logueamos los parámetros
    mlflow.log_params(rf_params)
    
    # Evaluamos en conjunto de entrenamiento
    y_train_pred = model.predict(X_train)
    train_f1 = f1_score(y_train.to_numpy().ravel(), y_train_pred, average='macro')
    mlflow.log_metric("train_f1", train_f1)
    
    # Evaluamos en conjunto de prueba
    y_test_pred = model.predict(X_test)
    test_f1 = f1_score(y_test.to_numpy().ravel(), y_test_pred, average='macro')
    mlflow.log_metric("test_f1", test_f1)
    
    # Configuramos tags para el experimento
    mlflow.set_tags(
        tags={
            "project": "Star Classification",
            "model_family": "sklearn",
            "model_type": "RandomForestClassifier",
            "feature_set_version": 1,
        }
    )
    
    # Guardamos el artefacto del modelo
    artifact_path = "model"
    
    signature = infer_signature(X_train, model.predict(X_train))
    
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="star_class_model_dev",
        metadata={"model_data_version": 1}
    )
    
    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)
    
    print(f"Train F1 Score: {train_f1:.4f}")
    print(f"Test F1 Score: {test_f1:.4f}")
    print(f"Model saved at: {model_uri}")
    
    # Guardamos el run_id para usarlo después
    current_run_id = mlflow.active_run().info.run_id


2025/08/07 01:11:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'star_class_model_dev' already exists. Creating a new version of this model...
2025/08/07 01:11:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: star_class_model_dev, version 4


Train F1 Score: 0.4960
Test F1 Score: 0.2977
Model saved at: s3://mlflow/1/00d352c403304299bdbe1f60446fb8bc/artifacts/model
🏃 View run Train RF at: http://localhost:5000/#/experiments/1/runs/00d352c403304299bdbe1f60446fb8bc
🧪 View experiment at: http://localhost:5000/#/experiments/1


Created version '4' of model 'star_class_model_dev'.


## Testeando el modelo

In [7]:
# Primero probamos con el modelo que ya tenemos en memoria
import numpy as np
test_data = [0.741830344520968, -1.3390058817968316, 0.9270742100032214, -0.022594692703017016, -0.020158498007864664,
             -0.32633765093444494, -0.16166633313766, 0.0006512010351584627, 0.7417317177335456, 0.9400630553605082, 
             0.2899351799963413, -0.9217758867440433, -0.788211578207859, -0.9217809160581861, -1.0088989161501674, 0.0013417780822043853]


In [8]:
# Usamos el modelo que ya está en memoria
prediction = model.predict(np.array(test_data).reshape([1, -1]))
print(f"Prediction with in-memory model: {prediction}")

Prediction with in-memory model: [0]


/home/jorge/miniconda3/envs/mlops-5/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [9]:
# Intentamos cargar el modelo desde diferentes fuentes
print("Intentando cargar el modelo...")
loaded = None

# Opción 1: Cargar desde el run artifacts
try:
    loaded = mlflow.sklearn.load_model(model_uri)
    prediction_loaded = loaded.predict(np.array(test_data).reshape([1, -1]))
    print(f"✓ Modelo cargado desde run artifacts: {prediction_loaded}")
except Exception as e:
    print(f"✗ Error cargando desde run artifacts: {e}")

# Opción 2: Cargar desde el Model Registry (más probable que funcione)
if loaded is None:
    try:
        # Cargamos desde el modelo registrado en desarrollo
        loaded = mlflow.sklearn.load_model("models:/star_class_model_dev/latest")
        prediction_loaded = loaded.predict(np.array(test_data).reshape([1, -1]))
        print(f"✓ Modelo cargado desde Model Registry (dev): {prediction_loaded}")
    except Exception as e:
        print(f"✗ Error cargando desde Model Registry (dev): {e}")

# Opción 3: Si todo falla, usar el modelo en memoria
if loaded is None:
    print("Usando modelo en memoria como fallback")
    loaded = model


/home/jorge/miniconda3/envs/mlops-5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Intentando cargar el modelo...


✗ Error cargando desde run artifacts: The following failures occurred while downloading one or more artifacts from s3://mlflow/1/00d352c403304299bdbe1f60446fb8bc/artifacts:
##### File model #####
An error occurred (404) when calling the HeadObject operation: Not Found


✓ Modelo cargado desde Model Registry (dev): [0]



/home/jorge/miniconda3/envs/mlops-5/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [10]:
### Registramos el modelo en producción 
#Realizamos el registro del modelo en MLflow para producción.

from mlflow import MlflowClient

client = MlflowClient()
name = "star_class_model_prod"
desc = "This is a star classifier model - Random Forest"

# Intentamos crear el modelo productivo (si ya existe, continuamos)
try:
    client.create_registered_model(name=name, description=desc)
    print(f"✓ Modelo registrado '{name}' creado")
except Exception as e:
    print(f"ℹ Model registry '{name}' ya existe: {e}")

# Guardamos como tag los hiper-parámetros en la versión del modelo
tags = model.get_params()
tags["model"] = type(model).__name__
tags["f1_score"] = str(test_f1)
tags["run_id"] = current_run_id

# Guardamos la versión del modelo
try:
    result = client.create_model_version(
        name=name,
        source=model_uri,
        run_id=current_run_id,
        tags=tags
    )
    
    # Y creamos como la versión con el alias de champion para poder levantarlo en nuestro
    # proceso de servicio del modelo on-line.
    client.set_registered_model_alias(name, "champion", result.version)
    
    print(f"✓ Model version {result.version} registered and set as champion")
    print(f"✓ Modelo disponible en: models:/{name}/champion")
    
    # Probamos cargar el modelo desde producción
    try:
        prod_model = mlflow.sklearn.load_model(f"models:/{name}/champion")
        prod_prediction = prod_model.predict(np.array(test_data).reshape([1, -1]))
        print(f"✓ Modelo productivo funciona correctamente: {prod_prediction}")
    except Exception as e:
        print(f"⚠ Error probando modelo productivo: {e}")
    
except Exception as e:
    print(f"✗ Error registering model version: {e}")
    print("Model was trained successfully but not registered in production registry")

2025/08/07 01:11:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: star_class_model_prod, version 1


ℹ Model registry 'star_class_model_prod' ya existe: RESOURCE_ALREADY_EXISTS: Registered Model (name=star_class_model_prod) already exists.
✓ Model version 1 registered and set as champion
✓ Modelo disponible en: models:/star_class_model_prod/champion
⚠ Error probando modelo productivo: INVALID_PARAMETER_VALUE: Invalid Model Version stage: champion. Value must be one of None, Staging, Production, Archived.
